#### Importação das bibliotecas necessárias

In [1]:
from datetime import datetime
import pandas as pd
import requests as r
#from conexao_sqlserver import cnxn, cursor
import pyodbc

In [2]:
credenciais = pd.read_csv('credenciais//credenciais.txt', sep=";")

# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
# server = tcp:localhost,1433 # to login in linux ubuntu
server = 'tcp:localhost,1433'
database = 'db_projeto_cotacao_online'
username = credenciais['user'][0] 
password = credenciais['senha'][0]
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

#### Requisição da URL que fornece os dados na fonte com atualização a cada 30 segundos

In [3]:
lista_cotacoes = ['USD-BRL','EUR-BRL','ARS-BRL','GBP-BRL','JPY-BRL','UYU-BRL','PYG-BRL','CLP-BRL']


lista_df_cotacoes = []

for i in lista_cotacoes:
    url = f"https://economia.awesomeapi.com.br/last/{i}"
    cotacao = r.get(url)
    cotacoes = cotacao.json()
    for resultado in cotacoes:
        nome = cotacoes[resultado]['name']
        hora_coleta_api = cotacoes[resultado]['create_date']
        valor_compra = float(cotacoes[resultado]['bid'])
        valor_venda = float(cotacoes[resultado]['ask'])
        valor_maximo = float(cotacoes[resultado]['high'])
        valor_minimo = float(cotacoes[resultado]['low'])

        lista_itens = [str(datetime.now())[:19], resultado, nome, hora_coleta_api, valor_compra, valor_venda, valor_maximo, valor_minimo]
        lista_df_cotacoes.append(lista_itens)

#### Integração com o SQL SERVER para armazenamento dos dados

In [4]:
query = """select 
codigo_cotacao + '|' + hora_disponibilidade_api as 'chave'
from tb_cotacao_moedas_hoje"""

cotacoes_ja_importadas_no_banco = []
consulta_sql = pd.read_sql_query(query, cnxn)
for i in consulta_sql.index:
    cotacoes_ja_importadas_no_banco.append(consulta_sql['chave'][i])

/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
for item in lista_df_cotacoes:
    chave = str(item[1]) + "|" + str(item[3])
    if chave in cotacoes_ja_importadas_no_banco:
        pass
    else:
        cursor.execute(f"""
        INSERT INTO tb_cotacao_moedas_hoje (importacao, codigo_cotacao, nome, hora_disponibilidade_api, valor_compra, valor_venda, valor_maximo, valor_minimo)
        VALUES ('{item[0]}',
                '{item[1]}',
                '{item[2]}',
                '{item[3]}',
                '{item[4]}',
                '{item[5]}',
                '{item[6]}',
                '{item[7]}'
                ) """)

        cnxn.commit()